# Table Of Content
---

1. ### [Data Processing](#data_processing)
    1. ### [User Data Processing](#user_data_processing)
    2. ### [Post Data Processing](#raw_post_data_processing)

    3. ### [Post Data Processing](#post_data_processing)

In [4]:
import pickle
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import deque
from datetime import *
import re
import glob
import os
from collections import defaultdict

<a name="data_processing"/>

# Data Processing

<a name="user_data_processing"/>

## User Data Processing

In [2]:
import os
path = '/root/CSC440_project/data_general'
def findfiles(path):
    return glob.glob(path)
working_directory = '/root/CSC440_project/data_general/tweets/*/'
tweets_path_list = findfiles(working_directory)
tweets_list = os.listdir('/root/CSC440_project/data_general/tweets')
# read location csv
location_to_join = pd.read_csv('/root/CSC440_project/data_general/locations_combined.csv', header = None, names = ['id','location'])
location_to_join['id'] = [str(i) for i in location_to_join['id']]
users = pd.read_csv(path+'/combined.csv')
users = users.rename(columns={'用户id':'id','性别': 'gender','微博数':'#tweets', '粉丝数': '#follower', '关注数': '#following','是否认证':'verified'})
users = users[['id','gender','#tweets','#follower','#following','verified']]
users = users.drop_duplicates()
users['id'] = [str(i) for i in users['id']]
user_id_lst = location_to_join['id']
users['content'] = np.zeros(len(users))
users.loc[users['id'].isin(user_id_lst), 'content'] = 1  
users_with_location = users.merge(location_to_join, how='inner', left_on='id', right_on='id')
users_with_content = users_with_location.drop_duplicates('id')
users_with_content = users_with_content.reset_index(drop=True)
users_with_content

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,gender,#tweets,#follower,#following,verified,content,location
0,6309921628,f,2902,344,403,False,1.0,河南 许昌
1,6007931743,m,11,20,163,False,1.0,江苏 南京
2,7471743898,m,5,10,265,False,1.0,其他
3,5701747600,f,3175,95113,1052,False,1.0,上海 杨浦
4,7577162125,m,25,3,190,False,1.0,山东 济南
...,...,...,...,...,...,...,...,...
574352,6505005302,f,36,21,239,False,1.0,其他
574353,1803190527,f,6243,1409,2069,False,1.0,浙江
574354,5343183078,m,418,148,336,False,1.0,北京 海淀
574355,6031048952,f,183,1,1116,False,1.0,海南 海口


<a name="raw_post_data_processing"/>

## Post Data Processing

In [3]:
rename_dict = {
    '视频url': 'reposter_device',  
    '原始图片url': 'reposting_time', 
    '位置': '#likes', 
    '日期': '#comments', 
    '工具': '#reposts', 
    '点赞数': 'repost_weibo_comment',
    '源用户id': 'source_user_id', 
    '源用户昵称': 'source_user_nickname', 
    '源微博原始图片url': 'source_weibo_post_time',
    '源微博视频url': 'source_weibo_device', 
    '源微博位置': '#source_weibo_likes', 
    '源微博日期': '#source_weibo_comments', 
    '源微博工具': '#source_weibo_reposts', 
    '源微博点赞数': 'source_weibo_content' 
}

rename_dict2 = {
    "正文": 'repost_weibo_comment',
    '日期': 'reposting_time',
    '工具': 'reposter_device',
    '点赞数': '#likes',
    '评论数':'#comments',
    '转发数': '#reposts',
    '源用户id': 'source_user_id',
    '源用户昵称': 'source_user_nickname',
    '源微博正文': 'source_weibo_content',
    '源微博日期': 'source_weibo_post_time',
    '源微博工具': 'source_weibo_device',
    '源微博点赞数': '#source_weibo_likes',
    '源微博评论数': '#source_weibo_comments',
    '源微博转发数': '#source_weibo_reposts'
    
}

string1 = '抱歉，由于作者设置，'
string2 = '该微博因被多人投诉'
string3 = '该账号因被投诉违反'
string4 = '该账号因被投诉'
string5 = '查看帮助： 网页链接'

censor_indications = [string1, string2, string2, string4, string5]

# resumption from break point?
def tweets_tackling(tweets_list, segment_index):
    censored_repost_list = deque()
    # /data_general/tweets/用户名/user_id.csv
    # count the number of censored reposts for each user
    # put then-censored reposts into one pandas dataframe
    
    for i in tqdm(tweets_list, desc="#posts", unit="posts", total=len(tweets_list), position=0):
        try:
            weibo_ = findfiles(i + '/*.csv')[0]
            id_ = os.path.splitext(os.path.basename(os.listdir(i)[0]))[0]
            user_weibo_df = pd.read_csv(weibo_)
            try:
                datetime.strptime(user_weibo_df['原始图片url'][0],'%a %b %d %X %z %Y')
                # needs rename
                user_weibo_df = user_weibo_df.drop(columns=['bid','id','正文','头条文章url','源微博@用户','源微博话题','源微博转发数', '源微博bid','源微博id','源微博正文', '源微博头条文章url','评论数', '转发数', '话题', '@用户', '是否原创','源微博评论数']).rename(columns=rename_dict)
                user_weibo_df = user_weibo_df.assign(
                    user_id = id_,
                    is_censored_repost = user_weibo_df['source_weibo_content'].map(lambda content: any((keyword in str(content) for keyword in censor_indications)))
                )
            except TypeError:
                user_weibo_df = user_weibo_df.drop(columns=['id','bid','头条文章url','原始图片url','视频url','源微博视频url','源微博位置','是否原创','源微博bid','话题','@用户','源微博@用户','位置','源微博id','源微博头条文章url','源微博原始图片url','源微博话题']).rename(columns=rename_dict2)
                user_weibo_df = user_weibo_df.assign(
                    user_id = id_,
                    is_censored_repost = user_weibo_df['source_weibo_content'].map(lambda content: any((keyword in str(content) for keyword in censor_indications)))
                )
            censored_repost_list.extend((row for _, row in user_weibo_df.iterrows()))
        except IndexError:
            print(i)

#     with open('__user_censornumber_%s.pkl' %thread_num , 'wb') as f:
#         pickle.dump(user_censornumber, f)

#     with open('__user_tweetsnumber_%s.pkl' %thread_num, 'wb') as f:
#         pickle.dump(user_tweetsnumber, f)
    censored_repost_df = pd.DataFrame(list(censored_repost_list))
    
    with open('__raw_repost_df_%s.pkl' % segment_index, 'wb') as f:
        pickle.dump(censored_repost_df,f)

In [8]:
def main():
    segment_length = 20000
    for segment_index in range(len(tweets_path_list)//segment_length + 1):
        path_segment = tweets_path_list[segment_index*segment_length: (segment_index+1)*segment_length]
        tweets_tackling(path_segment, segment_index)

In [ ]:
main()

<a name="post_number"/>

### Post Number and Censored Number

In [ ]:
censored_num_recorder = defaultdict(int)
collected_repost_num = defaultdict(int)
for i in range(len(tweets_path_list)//segment_length + 1):
    with open('__raw_repost_df_%s.pkl' %i, 'rb') as f:
        temp_raw_df = pickle.load(f)
    for key, values in defaultdict(int,temp_raw_df.groupby('user_id')['is_censored_repost'].agg('sum').to_dict()).items():
        censored_num_recorder[key] += values
    for key, values in defaultdict(int,temp_raw_df.groupby('user_id').size().to_dict()).items():
        collected_repost_num[key] += values
with open('user_censorship_num_recorder.pkl','wb') as f:
    pickle.dump(censored_num_recorder, f)
with open('user_collected_repost_num.pkl','wb') as f:
    pickle.dump(collected_repost_num, f)
    

### Before running the cell below, you have to check if the number of keys in the collected_posts matches the number of length of users_with_content. If there is no exact match, you can subset users_with_content with the IDs that are in the dictionary

In [ ]:
####################### Manual check here!!!



In [ ]:
users_with_content['#collected_posts'] = users_with_content['id'].map(lambda x: collected_repost_num[x])
users_with_content['#censored_posts'] = users_with_content['id'].map(lambda x: censored_num_recorder[x])

In [70]:
users_with_content.head()

,id,gender,#tweets,#follower,#following,verified,content,location
0,6309921628,f,2902,344,403,False,1.0,河南 许昌
1,6007931743,m,11,20,163,False,1.0,江苏 南京
2,7471743898,m,5,10,265,False,1.0,其他
3,5701747600,f,3175,95113,1052,False,1.0,上海 杨浦
4,7577162125,m,25,3,190,False,1.0,山东 济南


<a name="follower_count_percentile"/>

### Follower Count Percentile

In [3]:
users_with_content.sort_values('#follower', inplace=True)
df_len = len(users_with_content)

lesser_num = 0
cur_follower_num = 0
cur_count = 0
percentiles = deque()

for follower_num in users_with_content['#follower']:
    if follower_num == cur_follower_num:
        cur_count += 1
    else:
        percentile = (lesser_num) / df_len * 100
        percentiles.extend([percentile for _ in range(cur_count)])
        cur_follower_num = follower_num
        lesser_num += cur_count 
        cur_count = 1

percentile = (lesser_num) / df_len * 100
percentiles.extend([percentile for _ in range(cur_count)])

users_with_content["#follower_percentile_grouped"] = list(map(lambda x: x//10 + 1, percentiles))
users_with_content["#follower_percentile_grouped"] = users_with_content["#follower_percentile_grouped"].astype('int32')
users_with_content.tail()

,id,gender,#tweets,#follower,#following,verified,content,location,#censoship,#follower_percentile_grouped
426005,2803301701,m,135297,138009780,3061,True,1.0,北京,0,10
258474,1934183965,f,2318,158452899,183,True,1.0,北京,0,10
203828,5878659096,m,10368,181509693,2553,True,1.0,北京,0,10
505034,2016713117,f,15318,183042610,135,True,1.0,北京 海淀,0,10
76225,1642909335,f,5632,197923455,2931,True,1.0,北京,0,10


<a name="location_to_province"/>

### Location to Province

In [4]:
weird_prefixes = {"昵称", "认证"}

def to_province(location):
    location = location.split(' ')[0]
    return location if all((matcher not in location for matcher in weird_prefixes)) else '其他'
users_with_content['province'] = users_with_content['location'].map(to_province)
users_with_content['province']

536321    其他
278927    其他
405016    其他
234168    其他
151815    河南
          ..
426005    北京
258474    北京
203828    北京
505034    北京
76225     北京
Name: province, Length: 574357, dtype: object

<a name="post_number"/>

### Post Number and Censored Number

In [6]:
from collections import defaultdict
user_post_nums = defaultdict(int)
for i in tqdm(range(12)):
    with open('raw_repost_df_w_id_%d.pkl' %i, 'rb') as f:
        df = pickle.load(f)
    for uid in df['user_id']:
        user_post_nums[uid] += 1
users_with_content['#collected_posts'] = users_with_content['id'].map(lambda uid: user_post_nums[uid])
users_with_content.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:33<00:00,  7.77s/it]


,id,gender,#tweets,#follower,#following,verified,content,location,#censoship,#follower_percentile_grouped,province,#collected_posts
536321,2453175617,f,2,0,172,False,1.0,其他,0,1,其他,1
278927,2123046974,f,8,0,18,False,1.0,其他,0,1,其他,3
405016,1883592641,m,724,0,1215,False,1.0,其他,1,1,其他,58
234168,2639003040,m,32,0,51,False,1.0,其他,14,1,其他,62
151815,2045211094,m,15,0,48,False,1.0,河南,0,1,河南,4


### Province GDP

In [7]:
from collections import defaultdict

gdp_by_province = defaultdict(int) # default value for GDP is 0

gdp_by_province.update({
    "北京":23805,
    "上海":23277,
    "江苏":17121,
    "浙江":16358,
    "福建":15531,
    "广东":14223,
    "天津":13569,
    "湖北":10988,
    "重庆":10867,
    "山东":10811,
    "内蒙古":9977,
    "陕西":9239,
    "安徽":8703,
    "湖南":8681,
    "辽宁":8667,
    "海南":8323,
    "河南":8302,
    "四川":8229,
    "新疆":7721,
    "宁夏":7686,
    "江西":7682,
    "青海":6998,
    "西藏":6997,
    "云南":6950,
    "贵州":6828,
    "河北":6797,
    "山西":6735,
    "吉林":6577,
    "广西":6386,
    "黑龙江":5129,
    "甘肃":4624,
    "香港":46700,
    "台湾":28306,
    "澳门":38769,
})

users_with_content['province_gdp'] = users_with_content['province'].map(lambda p: gdp_by_province[p])
users_with_content.head()

,id,gender,#tweets,#follower,#following,verified,content,location,#censoship,#follower_percentile_grouped,province,#collected_posts,province_gdp
536321,2453175617,f,2,0,172,False,1.0,其他,0,1,其他,1,0
278927,2123046974,f,8,0,18,False,1.0,其他,0,1,其他,3,0
405016,1883592641,m,724,0,1215,False,1.0,其他,1,1,其他,58,0
234168,2639003040,m,32,0,51,False,1.0,其他,14,1,其他,62,0
151815,2045211094,m,15,0,48,False,1.0,河南,0,1,河南,4,8302


### Censored Ratio & renaming

In [8]:
users_with_content = users_with_content.rename(columns={
    'verified':"verification"
})
users_with_content['censored'] = users_with_content['#censored_posts'].map(lambda x: x > 0)
users_with_content['censored_ratio'] = users_with_content['#censored_posts']/users_with_content['#collected_posts']
users_with_content['censored_ratio'] = users_with_content['censored_ratio'].map(lambda x: x if x != float('inf') else 0)
users_with_content.head()

,id,gender,#tweets,#follower,#following,verification,content,location,#censored_posts,#follower_percentile_grouped,province,#collected_posts,province_gdp,censored,censored_ratio
536321,2453175617,f,2,0,172,False,1.0,其他,0,1,其他,1,0,False,0.000000
278927,2123046974,f,8,0,18,False,1.0,其他,0,1,其他,3,0,False,0.000000
405016,1883592641,m,724,0,1215,False,1.0,其他,1,1,其他,58,0,True,0.017241
234168,2639003040,m,32,0,51,False,1.0,其他,14,1,其他,62,0,True,0.225806
151815,2045211094,m,15,0,48,False,1.0,河南,0,1,河南,4,8302,False,0.000000


### Unused Fields Cleansing

In [9]:
users_with_content.drop([
    'content',
    'location'
], axis = 'columns', inplace=True, errors='ignore')
users_with_content.head()

,id,gender,#tweets,#follower,#following,verification,#censored_posts,#follower_percentile_grouped,province,#collected_posts,province_gdp,censored,censored_ratio
536321,2453175617,f,2,0,172,False,0,1,其他,1,0,False,0.000000
278927,2123046974,f,8,0,18,False,0,1,其他,3,0,False,0.000000
405016,1883592641,m,724,0,1215,False,1,1,其他,58,0,True,0.017241
234168,2639003040,m,32,0,51,False,14,1,其他,62,0,True,0.225806
151815,2045211094,m,15,0,48,False,0,1,河南,4,8302,False,0.000000


In [10]:
users_with_content

,id,gender,#tweets,#follower,#following,verification,#censored_posts,#follower_percentile_grouped,province,#collected_posts,province_gdp,censored,censored_ratio
536321,2453175617,f,2,0,172,False,0,1,其他,1,0,False,0.000000
278927,2123046974,f,8,0,18,False,0,1,其他,3,0,False,0.000000
405016,1883592641,m,724,0,1215,False,1,1,其他,58,0,True,0.017241
234168,2639003040,m,32,0,51,False,14,1,其他,62,0,True,0.225806
151815,2045211094,m,15,0,48,False,0,1,河南,4,8302,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426005,2803301701,m,135297,138009780,3061,True,0,10,北京,11,23805,False,0.000000
258474,1934183965,f,2318,158452899,183,True,0,10,北京,51,23805,False,0.000000
203828,5878659096,m,10368,181509693,2553,True,0,10,北京,51,23805,False,0.000000
505034,2016713117,f,15318,183042610,135,True,0,10,北京,54,23805,False,0.000000


In [11]:
with open('./final/user_df.pkl', 'wb') as f:
    pickle.dump(users_with_content, f)

<a name="post_data_processing"/>

## Post Data Processing

### Device Transform Definition

In [4]:
device_mapping = {
    'Apple':['iPhone','iPad','Mac'],
    'Web':['浏览器', '微博'],
    'Huawei':['Huawei','nova','华为','HUAWEI','nova','Harmony'],
    'Honor':['荣耀'],
    'XiaoMi':['小米', 'Redmi','红米'],
    'vivo':['vivo'],
    'OPPO':['OPPO'],
    'Samsung':['三星','Samsung'],
    'General Andoid':['android','Android'],
    'Realme':['realme','真我'],
    'IQOO':['iQOO'],
    'OnePlus':['一加','OnePlus']
}

def get_reposter_device(df):   
    def to_brand(client_name):
        if type(client_name) is not str:
            return 'NaN'
        for brand, devices in device_mapping.items():
            if any((device in client_name for device in devices)):
                return brand
        return 'other'
    
    return df['reposter_device'].map(to_brand).astype('string')

get_reposter_device(temp_df)

0    Apple
1    Apple
2    Apple
0     vivo
1     vivo
Name: reposter_device, dtype: string

### Censored Transform Definition

### Time Transform Definition

In [15]:
from datetime import datetime

def get_timestamp(df):
    def to_timestamp(s):
        if type(s) is not str:
            return None
        return datetime.strptime(s, '%a %b %d %X %z %Y').replace(hour=0, minute=0, second=0, microsecond=0, day=1).timestamp()
    return df['reposting time'].map(to_timestamp)

get_timestamp(temp_df)

0    1.627747e+09
1    1.606752e+09
2    1.588262e+09
0    1.627747e+09
1    1.627747e+09
Name: reposting time, dtype: float64

In [82]:
def get_timestamp2(df):
    def to_timestamp(s):
        if type(s) is not str:
            return None
        return datetime.strptime(s, '%a %b %d %X %z %Y').replace(hour=0, minute=0, second=0, microsecond=0, day=1).timestamp()
    return df['source_weibo_post_time'].map(to_timestamp)


### User Information Preparation

In [16]:
users_df = users_with_content.copy()
users_df = users_df.set_index('id')
users_df.head()

,gender,#tweets,#follower,#following,verification,#censored_posts,#follower_percentile_grouped,province,#collected_posts,province_gdp,censored,censored_ratio
id,,,,,,,,,,,,
2453175617,f,2,0,172,False,0,1,其他,1,0,False,0.000000
2123046974,f,8,0,18,False,0,1,其他,3,0,False,0.000000
1883592641,m,724,0,1215,False,1,1,其他,58,0,True,0.017241
2639003040,m,32,0,51,False,14,1,其他,62,0,True,0.225806
2045211094,m,15,0,48,False,0,1,河南,4,8302,False,0.000000


### Transformation Pipeline

In [18]:
def pipeline(df, partition):
    df = df.assign(
        general_device = get_reposter_device(df),
        repost_timestamp_month = get_timestamp(df)
        post_timestamp_month = get_timestamp2(df)
    )
    df.drop([
        "source_user_nickname",
        "source_user_id",
        "reposter_device",
        "source_weibo_device",
        "reposting time",
        "#likes",
        "#comments",
        "#reposts",
        "#source_weibo_likes",
        "#source_weibo_comments",
        "source_weibo_reposts"
    ], axis = 'columns', inplace=True)
    df = df.merge(users_df, left_on='user_id', right_on='id', right_index=True, suffixes=('_post', '_user'))
    with open('final/post_full_df_%d.pkl' % partition, 'wb') as f_out:
        pickle.dump(df, f_out)
        
    df.drop([
        "repost_weibo_comment",
        "source_weibo_content"
    ], axis = 'columns', inplace=True)
    return df

# testing
pipeline(temp_df, 10086)

,user_id,general_device,censored_post,timestamp_month,gender,#tweets,#follower,#following,verification,#censored_posts,#follower_percentile_grouped,province,#collected_posts,province_gdp,censored_user,censored_ratio
0,5711341129,Apple,False,1.627747e+09,f,48,193,394,False,0,7,其他,3,0,False,0.000000
1,5711341129,Apple,False,1.606752e+09,f,48,193,394,False,0,7,其他,3,0,False,0.000000
2,5711341129,Apple,False,1.588262e+09,f,48,193,394,False,0,7,其他,3,0,False,0.000000
0,7412051152,vivo,False,1.627747e+09,m,129,1,38,False,1,1,其他,52,0,True,0.019231
1,7412051152,vivo,False,1.627747e+09,m,129,1,38,False,1,1,其他,52,0,True,0.019231


In [19]:
# Transform Begin

num_dfs = len(tweets_path_list)//20000 + 1

no_content_post_dfs = []

for i in tqdm(range(num_dfs)):
    with open('__raw_repost_df_%s.pkl' %i, 'rb') as f:
        df = pickle.load(f)
    no_content_post_df = pipeline(df, i)
    no_content_post_dfs.append(no_content_post_df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [08:23<00:00, 41.94s/it]


In [4]:
# Run this in case memory outage and no_content_post get stucked

# no_content_post_dfs = []
# for i in tqdm(range(12)):
#     with open('final/post_full_df_%d.pkl' %i, 'rb') as f:
#         df = pickle.load(f)
#     df.drop([
#         "repost_weibo_comment",
#         "source_weibo_content"
#     ], axis = 'columns', inplace=True)
#     no_content_post_dfs.append(df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:49<00:00,  4.14s/it]


In [8]:
with open('final/post_no_content_df.pkl', 'wb') as f_out:
    pickle.dump(pd.concat(no_content_post_dfs), f_out)